### Basics Imports

In [1]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')
import numpy as np
import pandas as pd
import sys

10/11/2023 02:10:25 AM-INFO: NumExpr defaulting to 8 threads.


### TensorNetworks imports

In [2]:
sys.path.append("../")
import tensornetworks as tn
import tn_quantum_circuits as tnqc
import gates as gt
from tensornetworks import contract_indices

### Imports QLM

In [3]:
import qat.lang.AQASM as qlm
from qat.qpus import PyLinalg
qpu_p = PyLinalg()
from qlm_stuff import proccess_qresults

In [4]:
def apply_2qubit_gates(qubits, gates):
    """
    Executes product of tensor with a gate
    -o-o-o-o-o-..o-o-
     |   |   |     |
    """
    new_qubits = [0 for i in qubits]
    left = qubits[0]
    for i in range(1, len(qubits)):
        right = qubits[i]
        gate = gates[i-1]
        #new_qubits[i-1], left = phase_change(left, right, gate)
        new_qubits[i-1], left = tnqc.apply_2qubit_gate(left, right, gate)

    new_qubits[-1], new_qubits[0] = tnqc.apply_2qubit_gate(
        left, new_qubits[0], gates[-1])
     #new_qubits[-1], new_qubits[0] = phase_change(left, new_qubits[0], gates[-1])
    return new_qubits

In [5]:
def get_angles(depth):
    theta = np.pi/4.0
    delta_theta = theta / (depth + 1)
    angles = []
    for i in range(depth):
        angles.append([(2 * i + 1) * delta_theta, (2 * i + 2) * delta_theta])
    return angles     

In [6]:
get_angles(4)

[[0.15707963267948966, 0.3141592653589793],
 [0.47123889803846897, 0.6283185307179586],
 [0.7853981633974483, 0.9424777960769379],
 [1.0995574287564276, 1.2566370614359172]]

In [7]:
def ansatz(nqubits, depth, angles):
    # Intitial State
    zeroket = np.zeros((1, 2, 1))
    zeroket[0][0][0] = 1
    zeroket = zeroket.astype(complex)
    #Initial State
    mps_ = [zeroket] * nqubits
    for depth_ in range(depth):
        # First Layer
        gates = [gt.x_rotation(angles[depth_][0]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
        ent_gates = [gt.controlz() for i in mps_]
        mps_ = apply_2qubit_gates(mps_, ent_gates)
        gates = [gt.z_rotation(angles[depth_][1]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
    return mps_

In [8]:
def ansatz_qlm(nqubits, depth, angles):
    qprog = qlm.Program()
    qbits = qprog.qalloc(nqubits)
    for d_ in range(0, depth):
        for i in range(nqubits):
            qprog.apply(qlm.RX(angles[d_][0]), qbits[i])
        for i in range(nqubits-1):
            qprog.apply(qlm.Z.ctrl(), qbits[i], qbits[i+1])    
        qprog.apply(qlm.Z.ctrl(), qbits[nqubits-1], qbits[0])
        for i in range(nqubits):
            qprog.apply(qlm.RZ(angles[d_][1]), qbits[i])    
    circ = qprog.to_circ()
    #%qatdisplay circ
    job = circ.to_job()
    state = qpu_p.submit(job)
    pdf = proccess_qresults(state, nqubits)
    pdf.reset_index(drop=True, inplace=True)
    return pdf, circ  

In [9]:
depth = 3
nqubits = 4 
mps = ansatz(nqubits, depth, get_angles(depth))
pdf_zalo = tnqc.get_state_from_mps(mps)

In [10]:
pdf, c= ansatz_qlm(nqubits, depth,  get_angles(depth))

10/11/2023 02:10:27 AM-INFO: Generating circuit.
10/11/2023 02:10:27 AM-INFO: Instantiating a Linker.
10/11/2023 02:10:27 AM-INFO: Linking libraries to circuit.
10/11/2023 02:10:27 AM-INFO: Found 0 ancillae
10/11/2023 02:10:27 AM-INFO: New batch of length 1 submitted
10/11/2023 02:10:27 AM-INFO: Compiling a batch of length 1
10/11/2023 02:10:27 AM-INFO: Starting compilation...
10/11/2023 02:10:27 AM-INFO: Returning compiled batch.
10/11/2023 02:10:27 AM-INFO: Resource management is not available, passing through.
10/11/2023 02:10:27 AM-INFO: Running jobs...
10/11/2023 02:10:27 AM-INFO: Done
10/11/2023 02:10:27 AM-INFO: Post processing a list of results of length 1
10/11/2023 02:10:27 AM-INFO: Wrapping results using 1 quantum registers


In [11]:
%qatdisplay c --svg

In [12]:
np.isclose(pdf["Amplitude"], pdf_zalo["Amplitude"]).all()

True

In [13]:
[mps_.shape for mps_ in mps]

[(16, 2, 8), (8, 2, 16), (16, 2, 16), (16, 2, 16)]

### Testing  Isometry

In [14]:
# Identitidad: U.T @ U
tensor = mps[3]
iso = tn.contract_indices(tensor, tensor.conj(), [0, 1], [0, 1])
np.isclose(iso, np.eye(iso.shape[0])).all()

True

In [15]:
#Proyector U @U.T
tensor = mps[3]
iso = tn.contract_indices(tensor, tensor.conj(), [2], [2])
projector = iso.reshape(np.product(iso.shape[:2]), np.product(iso.shape[:2]))
np.isclose(projector @ projector, projector).all()

True

In [16]:
# Identitidad: U.T @ U
tensor = mps[0]
iso = tn.contract_indices(tensor, tensor.conj(), [1, 2], [1, 2])
np.isclose(iso, np.eye(iso.shape[0])).all()

False

In [17]:
amp = np.array(pdf_zalo["Amplitude"])
amp = amp.reshape(amp.shape[0], 1)
rho0 = amp @ amp.conj().T

In [18]:
# Computing Density Matrix
amp = np.array(pdf_zalo["Amplitude"])
amp = amp.reshape(tuple([2 for i in range(nqubits)]))
rho1 = tn.reduced_matrix(amp, [0, 1, 2, 3], [])

In [19]:
np.isclose(rho0, rho1).all()

True

In [20]:
non_contracted = 1
tensor = mps[:non_contracted]
b = tnqc.compose_mps(mps[non_contracted:], False)

In [21]:
b.shape

(8, 2, 2, 2, 16)

In [23]:
[t.shape for t in tensor]

[(16, 2, 8)]

In [ ]:
contract_indices(b)

In [ ]:
b.shape

In [ ]:
np.isclose(a.reshape(2**4, 2**4), rho0).all()

In [ ]:
forrho = [mps[0]]a
b = tnqc.compose_mps(forrho)
a = contract_indices(np.conj(mps[0]),mps[0],  [0, 2], [0, 2])

In [ ]:
a

In [ ]:
mps[0].shape

In [ ]:
tn.reduced_matrix(amp, [0], [1, 2, 3])

In [ ]:
len(forrho)

In [ ]:
contract_indices?

In [ ]:
pdf_zalo["Amplitude"]

In [ ]:
a = tn.compose_mps(mps)

In [ ]:
b = a.reshape(np.prod(a.shape))

In [ ]:
b

In [ ]:
a = a.reshape(a.shape[0], -1, a.shape[-1])

In [ ]:
a.shape

In [ ]:
a.shape

In [ ]:
opa = contract_indices(a.conj(), a,  [0, 2], [0, 2])

In [ ]:
opa

In [ ]:
rho1

In [ ]:
amp.shape

In [ ]:
a = tn.compose_mps(mps)

In [ ]:
a.ndim

In [29]:
mps

[array([[[ 1.80453727e-22-1.20575326e-22j,
          -5.87520243e-03-1.49635574e-02j,
           1.58660975e-01+3.20535386e-01j,
           2.94370813e-01-2.18861784e-01j,
           4.67889678e-02+4.76095683e-01j,
           4.08299596e-01-2.87399883e-01j,
           2.40854101e-01+2.51390888e-01j,
          -2.92758826e-01+2.23957074e-01j],
         [-2.41674119e-01+1.67953925e-01j,
           1.03743731e-01+2.75461518e-01j,
          -7.83436579e-02+3.75622341e-01j,
          -3.83402488e-01-2.47258792e-02j,
           4.13594546e-01-4.88633439e-02j,
           2.47946110e-01+3.29527622e-01j,
           2.47152672e-01-1.93858943e-01j,
          -1.59460876e-01-2.49525016e-01j]],
 
        [[ 6.49208788e-24-4.33787444e-24j,
           1.60226192e-02-2.21496293e-02j,
          -3.62009321e-01+3.42648691e-01j,
           3.87403609e-01+2.87976314e-01j,
           3.67994312e-01+1.13847052e-01j,
          -6.89146820e-02-3.50867792e-01j,
          -2.39635245e-01-2.42380260e-01j,
      

In [30]:
def compose_from_right(mps):
    mps.reverse()
    tensor = mps[0]
    tensor_out = contract_indices(tensor, tensor.conj(), [1, 2], [1, 2])
    for tensor in mps[1:]:
        tensor_out = contract_indices(tensor, tensor_out, [2], [0])
        tensor_out = contract_indices(tensor_out, tensor.conj(), [1, 2], [1, 2])
    mps.reverse()
    return tensor_out
        

In [33]:
compose_from_right(mps)

array([[ 1.56693652e+00-9.71445147e-17j,  2.29019764e+00-2.70216324e-02j,
         3.49207651e-01-1.55176502e+00j,  3.94054502e-01-6.98021225e-02j,
        -4.25876020e-02-2.67929055e-02j, -4.54315272e-03+5.75411389e-02j,
         1.33466798e-02-9.09616737e-03j, -8.57650654e-02+7.63764060e-02j,
        -5.03219881e-18+2.69051805e-18j,  1.62496530e-18+4.24362353e-18j,
         6.22471979e-19-2.74638071e-19j,  2.59885022e-19-6.73425153e-19j,
        -1.71613511e-19+9.42993330e-20j,  1.27024818e-19+6.70190996e-20j,
        -2.06191867e-20-1.75856703e-20j,  6.40958646e-20-6.12441750e-21j],
       [ 2.29019764e+00+2.70216324e-02j,  4.41998318e+00+4.16333634e-16j,
         3.78748442e-03-2.88641228e+00j,  7.45495757e-01-5.14111103e-02j,
        -8.74266298e-02-3.15836015e-02j,  2.27822819e-02+9.96105388e-02j,
         2.09423073e-02-5.57727866e-03j, -1.25885498e-01+1.45936540e-01j,
        -1.62818146e-17+2.06299637e-18j, -5.03197525e-18+4.85741785e-18j,
         4.00342408e-18+1.15878767e-1

In [35]:
non_contracted = 1
tensor = mps[:non_contracted]
b = compose_from_right(mps[non_contracted:])

In [37]:
b.shape

(8, 8)

In [49]:
def opa(mps):
    tensor = mps[0]
    tensor_out = contract_indices(tensor.conj(), tensor, [0], [0])
    for tensor in mps[1:]:
        tensor_out = contract_indices(tensor, tensor_out, [0], [1])
        tensor_out = contract_indices(tensor_out.conj(), tensor, [tensor_out.ndim-1], [0])
    return tensor_out

In [50]:
c = opa(tensor)

In [51]:
contract_indices(c, b, [1, 3], [0, 1])

array([[ 8.        +4.67411309e-17j, -0.02567216+2.23406180e-01j],
       [-0.02567216-2.23406180e-01j,  8.        +2.72492458e-16j]])

In [52]:
# Computing Density Matrix
amp = np.array(pdf_zalo["Amplitude"])
amp = amp.reshape(tuple([2 for i in range(nqubits)]))
rho1 = tn.reduced_matrix(amp, [0], [1, 2, 3])

In [53]:
rho1

array([[0.60737583+6.61544034e-18j, 0.00869067-1.14587365e-01j],
       [0.00869067+1.14587365e-01j, 0.39262417+4.50278658e-18j]])

In [45]:
c.shape

(2, 8, 2, 8)

In [24]:
def density_matrix(mps):
    tensor = mps[0]
    tensor_out = contract_indices(tensor, tensor.conj(), [0], [0])
    for tensor in mps[1:-1]:
        print(tensor_out.shape)
        print(tensor.shape)
        print(tensor_out.ndim)
        tensor_out = contract_indices(tensor, tensor_out, [0], [1])
        tensor_out = contract_indices(tensor_out, tensor.conj(), [tensor_out.ndim-1], [0])
    tensor = mps[-1]
    tensor_out = contract_indices(tensor, tensor_out, [0], [1])
    tensor_out = contract_indices(tensor_out, tensor.conj(), [tensor_out.ndim-1, 1], [0, 2])
    return tensor_out
        
        
    

In [25]:
opa = density_matrix(mps)

(2, 8, 2, 8)
(8, 2, 16)
4
(2, 16, 2, 2, 2, 16)
(16, 2, 16)
6


In [26]:
rho_mps = opa.reshape(2**(opa.ndim //2), 2**(opa.ndim //2))

In [27]:
rho_mps

array([[ 1.01782269e+00-1.11022302e-16j, -1.49880108e-15-1.38777878e-16j,
         6.73072709e-16-1.99840144e-15j,  6.93889390e-17-4.44089210e-16j,
         2.46431409e-02+1.02075232e-02j,  8.88178420e-16+7.63278329e-17j,
         1.46410661e-15-4.15813217e-15j, -1.70696790e-15-1.41553436e-15j,
        -2.14106823e-01-1.15282503e-01j, -2.94447626e-15+2.85882429e-15j,
        -1.94289029e-15+2.35922393e-16j,  2.28983499e-15+1.90125693e-15j,
        -1.15567643e-02+4.04503925e-02j,  4.71844785e-16+4.56232274e-16j,
         1.27675648e-15+4.00374178e-15j,  3.56659147e-15-7.28583860e-16j],
       [-2.14106823e-01+1.15282503e-01j,  3.54577478e-15+2.01227923e-15j,
         1.49880108e-15+6.01602101e-15j,  3.25434124e-15+2.55004351e-16j,
        -2.04308805e-02+3.67746133e-02j, -1.17961196e-15+2.57519700e-15j,
         3.46944695e-16+7.17481630e-15j,  1.06858966e-15+2.83106871e-15j,
         9.82177308e-01-6.93889390e-18j,  9.36750677e-16+3.88578059e-16j,
        -8.74300632e-16+1.42941214e-1

In [28]:
rho0

array([[ 0.17365405+0.00000000e+00j,  0.06588595-7.67779379e-02j,
         0.06588595-7.67779379e-02j, -0.09560148+3.42396487e-02j,
         0.06588595-7.67779379e-02j,  0.05288058-1.23854633e-01j,
        -0.09560148+3.42396487e-02j,  0.07065793-2.92788947e-02j,
         0.06588595-7.67779379e-02j, -0.09560148+3.42396487e-02j,
         0.05288058-1.23854633e-01j,  0.07065793-2.92788947e-02j,
        -0.09560148+3.42396487e-02j,  0.07065793-2.92788947e-02j,
         0.07065793-2.92788947e-02j,  0.03592546-1.85639136e-02j],
       [ 0.06588595+7.67779379e-02j,  0.05894368+0.00000000e+00j,
         0.05894368+4.68375339e-17j, -0.05141051-2.92775938e-02j,
         0.05894368+1.57859836e-16j,  0.07482342-2.36114153e-02j,
        -0.05141051-2.92775938e-02j,  0.03975339+2.01314202e-02j,
         0.05894368-3.46944695e-18j, -0.05141051-2.92775938e-02j,
         0.07482342-2.36114153e-02j,  0.03975339+2.01314202e-02j,
        -0.05141051-2.92775938e-02j,  0.03975339+2.01314202e-02j,
         

In [ ]:
np.isclose(rho0, rho_mps)

In [ ]:
opa.ndim /2

In [ ]:
opa.size